In [ ]:
!pip install rdflib
!pip install pandas

Get Relative Path

In [1]:
import os

# Get the directory of the current script
base_dir = os.getcwd()
try:
    if folder:
        base_dir = os.path.join(base_dir, folder)
except:
    pass

Utililtiy Functiions

In [2]:
import uuid

def generate_unique_uri(base_uri):
    unique_identifier = str(uuid.uuid4())
    return URIRef(f"{base_uri}{unique_identifier}")

In [3]:
from rdflib import *
from hashlib import sha256


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Create Source Graph

Namespaces

In [4]:
schema = Graph()
schema.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
schema.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

## Load Data

In [5]:
import pandas as pd
data_path = os.path.join(base_dir, '../../DataProductLayer/DataProduct/Data/Explotation/UPENN-GBM_clinical_info_v2.1.csv')
df = pd.read_csv(data_path)
df.info()

### OWNER

In [7]:
metadata_path = os.path.join(base_dir, '../../DataProductLayer/DataProduct/Metadata/explotaiton.csv')

In [8]:
meta_df = pd.read_csv(metadata_path)
meta_dp = meta_df.loc[meta_df['name'] == "UPENN-GBM_clinical_info_v2"]
owner = meta_dp.owner.values[0]


### NAME

In [9]:
dataset_name = meta_dp.name.values[0]
dataset_name

'UPENN-GBM_clinical_info_v2'

add triples

In [10]:
# Name
schema.add((abox[dataset_name], RDF.type, dcat.Dataset))
#identifier
identifier = sha256(dataset_name.encode('utf-8')).hexdigest()
schema.add((abox[dataset_name],dcterms.identifier, Literal(identifier)))
#owner
schema.add((abox[dataset_name], tbox.owner, Literal(owner)))


<Graph identifier=N65b0dd6ecf454d11b619802f73858263 (<class 'rdflib.graph.Graph'>)>

### ATTRIBUTES


In [11]:
for column in df.columns:
    # Create RDF triple for Attribute
    schema.add((abox[column], RDF.type, tbox.Attribute))
    # Create RDF triple for Attribute
    schema.add((abox[dataset_name], tbox.hasAttribute, abox[column]))
    # rdf for the dataproperty
    schema.add((abox[column], tbox.attribute, Literal(column)))




## DatasetTypeTemplate

### Format

In [12]:
format = meta_dp.format.values[0]
format

'application/csv'

In [13]:
# triple
schema.add((abox.Tabular, RDF.type, tbox.DatasetTypeTemplate))

# format
schema.add((abox.Tabular, dcterms['format'], Literal(format)))  # Correct usage of the namespace

# addDTT
schema.add((abox[dataset_name], tbox.hasDTT, abox.Tabular))


<Graph identifier=N65b0dd6ecf454d11b619802f73858263 (<class 'rdflib.graph.Graph'>)>

### TechnologyAspects

In [14]:
#triple
schema.add((abox.TA1, RDF.type, tbox.TechnologyAspects))

#  addTA
schema.add((abox[dataset_name], tbox.hasTA, abox.TA1))


#TYPE

acces_uri = generate_unique_uri(abox)

schema.add((abox.TA1, tbox.typeAcces, acces_uri))


schema.add((acces_uri, RDF.type, tbox.Acces))
schema.add((acces_uri, RDFS.label, abox.Static))

# PATH
schema.add((acces_uri, tbox.path, Literal(data_path)))


<Graph identifier=N65b0dd6ecf454d11b619802f73858263 (<class 'rdflib.graph.Graph'>)>

# SAVE SOURCE_GRAPH

In [15]:
rdf_format = "turtle"
schema.serialize(destination=os.path.join(base_dir, 'Source_Graph_DP1.ttl'), format=rdf_format)

<Graph identifier=N65b0dd6ecf454d11b619802f73858263 (<class 'rdflib.graph.Graph'>)>